## Movie Recommender System

### KNNeighbors and KMeans

In [14]:
import pandas as pd
from scipy import linalg as la
import numpy as np
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from scipy.sparse import linalg as spla
from time import time

### Notes
 - Create a graph showing the distribution of average ratings for all users
 - Maybe another graph grouped by clusters

In [2]:
#Benjamin's path: path = "/Users/Armen/Desktop/SpringDataProject/"
#Ben C's path:
path = "/Users/benchristensen/Desktop/Recommneder_System_Project/ml-20m/"
ratings = pd.read_csv(path + "ratings.csv", nrows = 10000)

In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580
5,1,112,3.5,1094785740
6,1,151,4.0,1094785734
7,1,223,4.0,1112485573
8,1,253,4.0,1112484940
9,1,260,4.0,1112484826


In [4]:
#Create variable "count" for number of ratings for each movie
ratings = ratings.merge(ratings.groupby("movieId").size().reset_index(name='count'), how='right', on='movieId')

In [5]:
#Drop any movies that have 17 or fewer ratings
ratings = ratings[ratings["count"]>17].sort_values(['userId', 'movieId'])

In [6]:
#Find the minimum number of movies reviewed by a user
reviews_per_user = ratings.groupby("userId").userId.apply(lambda x: len(x))
reviews_per_user.describe().min()

1.0

## inherit from sklearn?
 - Scoring Accuracy:
 - predict for every movie of the user (binary recommend/do not recommend)
 - that will be your 'x test'
 - create a 'y test' that gives if the user liked each movie or not
 - percent correctly predicted.

## KMeans - Clustering Users

Method for predicting user $i$'s movie rating for movie $j$:

$P_{i, j}:$ Prediction for user $i$'s rating of movie $j$

$\bar{u}_i:$ User $i$'s average movie rating

$\bar{m}_j:$ Cluster's average rating for movie $j$

$\bar{c}:$ Cluster's average movie rating

$C:$ Cluster corresponding to user $i$

$M:$ All movies

$\mathbb{1}(r_{i,j}):$ A function indiciating if user $i$ has rated movie $j$ 

$\gamma_j : \frac{\sum_{i\in C}{\mathbb{1}(r_{i,j})}}{\sum_{i\in C, k\in M}\mathbb{1}(r_{i,k})}$

$$P_{i, j} = \bar{u}_i + (\bar{m}_j - \bar{c})*\frac{\gamma_j}{\max_{k\in M}{\gamma_k}} $$



In [92]:
def KMeans_Predict(model, df):
    """
    Use a fitted KMeans clustering model to predict a user's movie rating
    
    Parameter:
        model (): Fitted KMeans Model
        df (Sparse DataFrame): user-movie rating sparse dataframe
    
    Returns:
        df (DataFrame): New Dataframe with NaNs filled with predicted ratings
        update_df (DataFrame): Dataframe for testing that includes predictions rather
                              than any actual ratings.
    """
    #Assign clusters
    df['cluster'] = model.labels_
    #Find the proportion of ratings given for each movie by cluster
    share_df = df.groupby(['cluster']).count()/df.groupby(['cluster']).count().sum(1).values.reshape((-1,1)) #SHARE
    #Find the average rating for each movie by cluster
    cluster_avgs_df = df.groupby(["cluster"]).mean()  #m_bar
    #Define the average movie rating over all movies by user
    df['avg'] = df.mean(1)                            #u_bar
    #Find the average movie rating over all movies by cluster
    cluster_avgs = df.groupby(["cluster"]).mean().avg #c_bar
    #Calculate the adjustment for predicting purposes
    diff_df = (cluster_avgs_df - cluster_avgs.values.values.reshape((-1,1)))*(share_df/share_df.max(0))
    #If no one in cluster has seen the movie, adjustment is zero
    diff_df = diff_df.fillna(0)
    #Make the diff_df same size as df
    update_df = df[['cluster']].reset_index().merge(diff_df, how='left', on='cluster').set_index('index')
    #Add the adjustment to each user's avg movie rating
    update_df = update_df.iloc[:,1:].apply(lambda x: x + df.avg)
    #Fill the NaNs with the predictions
    df.update(update_df, overwrite=False)
    
    return df.iloc[:,:-2], update_df

def Test_KMeans(updated, predictions):
    #Find the sum of squared residuals REPLACE WITH ANOTHER LOSS METRIC? SOFTMAX??
    return ((updated - predictions)**2).sum().sum() / len(updated)

def standardize(df):
    """
    Change Predictions to standard movie rating format:
        Only takes values 0.5, 1, 1.5, ..., 5.0
        
    Parameters:
        df(Sparse Dataframe): user-movie rating dataframe
        
    Returns:
        (Sparse Dataframe): Standardized user-movie rating dataframe
    """
    #Only pass in columns of movie_IDs to standardize movie ratings for
    #Wouldn't want to change clusters
    clipped = df.clip(.5, 5)
    return clipped.apply(lambda x: round(2*x)/2)
    

In [22]:
path = "/Users/benchristensen/Desktop/Recommneder_System_Project/ml-20m/"
ratings = pd.read_csv(path + "ratings.csv", nrows = 10000)
#Generate user-movie matrix with ratings as its values
user_c = CategoricalDtype(sorted(ratings.userId.unique()), ordered=True)
movie_c = CategoricalDtype(sorted(ratings.movieId.unique()), ordered=True)

row = ratings.userId.astype(user_c).cat.codes
col = ratings.movieId.astype(movie_c).cat.codes
sparse_matrix = csr_matrix((ratings['rating'], (row, col)), \
                           shape=(user_c.categories.size, movie_c.categories.size))

In [23]:
#Create sparse dataframe from user-movie matrix
dfs = pd.SparseDataFrame(sparse_matrix, \
                         index=user_c.categories, \
                         columns=movie_c.categories) #,
                         #default_fill_value=2.5)
#Initialize KMeans model
##Kmeans can't be performed in SciKitLearn on data with missing values
##So we fill NA with 2.5 to find the clusters
KM_model = KMeans(n_clusters=5).fit(dfs.fillna(2.5))
KM_model.labels_
df = dfs.copy()

In [102]:
KM_model = KMeans(n_clusters=5).fit(dfs.fillna(2.5))
KM_model.labels_
df = dfs.copy()

In [103]:
new, test = KMeans_Predict(KM_model, df)
mse = Test_KMeans(standardize(new), standardize(test))

In [104]:
print("mean squared error:", mse)

mean squared error: 59.69505494505494


## KNN with cosine similarity

In [15]:
path = "/Users/benchristensen/Desktop/Recommneder_System_Project/ml-20m/"
ratings = pd.read_csv(path + "ratings.csv", nrows = 10000)
#Generate user-movie matrix with ratings as its values
user_c = CategoricalDtype(sorted(ratings.userId.unique()), ordered=True)
movie_c = CategoricalDtype(sorted(ratings.movieId.unique()), ordered=True)

row = ratings.userId.astype(user_c).cat.codes
col = ratings.movieId.astype(movie_c).cat.codes
sparse_matrix = csr_matrix((ratings['rating'], (row, col)), \
                           shape=(user_c.categories.size, movie_c.categories.size))
#Create sparse dataframe from user-movie matrix
dfs = pd.SparseDataFrame(sparse_matrix, \
                         index=user_c.categories, \
                         columns=movie_c.categories) #,
                         #default_fill_value=2.5)

Cosine Similarity:
$$sim(u, v) = \frac{\sum_i{\hat{r}_{ui}\hat{r}_{vi}}}{\sqrt{\sum_i{\hat{r}_{ui}^2}}\sqrt{\sum_i{\hat{r}_{vi}^2}}}$$

In [16]:
def cosine_similarity(u, v):
    """
    Find the cosine similarity between two users given their 
        movie ratings.
        
    Parameters:
        u (Series): user 'u' movie ratings
        v (Series): user 'v' movie ratings
        
    Returns:
        (float): cosine similarity between u and v
    """
    #Demean the ratings
    #    then fill NaN values with 0
    #    this penalizes not seeing the same movies
    u_hat = (u-u.mean()).fillna(0)
    v_hat = (v-v.mean()).fillna(0)
    
    return np.sum(u_hat*v_hat)/(np.sqrt(np.sum(u_hat**2))*np.sqrt(np.sum(v_hat**2)))
    

Prediction using cosine similarity (Pham et. al):

$P_{i, j}:$ Prediction for user $i$'s rating of movie $j$

$r_{k,j}:$ User $k$'s rating of movie $j$ (Equal to $\bar{r}_k$ if user $k$ hasn't rated movie $j$)

$\bar{r}_i:$ User $i$'s average movie rating

$w_{i,n}:$ Cosine similarity between user $i$ and user $k$

$N_i:$ The set of user $i$'s neighbors

$$P_{i,j} = \bar{r}_i + \frac{\sum_{k\in N}{(r_{k,j}-\bar{r}_k)w_{i,k}}}{\sum_{k\in N}{\mid w_{i,k}\mid}}    $$


In [125]:
def KNN_Predict(df, k):
    """
    Peform K-Nearest Neighbors and use it to predict movie ratings
    
    Parameter:
        df (Sparse DataFrame): user-movie rating sparse dataframe
        k (int): Hyperparameter - the number of neighbors
    
    Returns:
        df (DataFrame): New Dataframe with NaNs filled with predicted ratings
        update_df (DataFrame): Dataframe for testing that includes predictions rather
                              than any actual ratings.
    """
    update_df = df.copy().to_dense()
    for user_id in df.index:
        user = df.loc[user_id]
        #Find the cosine similarity for 'user' with every other user
        similarities = df.apply(lambda row: cosine_similarity(row, user), axis=1)
        similarity_df = pd.DataFrame(data={"similarity": similarities, "idx": similarities.index})
        similarity_df.sort_values("similarity", ascending = False, inplace=True)
        #Find the k nearest neighbors
        neighbors = similarity_df.iloc[1:k+1].idx.values
        #Predict movie rating using eq. (4) from Pham et. al paper
        predict = user.mean() + ((df.loc[neighbors] - df.loc[neighbors].mean(1).values.reshape((-1,1))) * similarity_df.iloc[1:k+1]["similarity"].values.values.reshape((-1,1))).sum()/similarity_df.iloc[1:k+1]["similarity"].abs().sum()  
        update_df.loc[user_id] = predict
    
    df.update(update_df, overwrite=False)
    return df, update_df

In [90]:
df = dfs.copy()

In [126]:
new, test = KNN_Predict(df, 5)

In [130]:
mse = Test_KMeans(standardize(new), standardize(test))
print("mean squared error:", mse)

mean squared error: 94.4532967032967


In [133]:
mse = Test_KMeans(new, test)
print("mean squared error:", mse)

mean squared error: 91.78305731559963


## Selecting Hyperparameters